In [23]:
import sys
import zmq
import json
import pandas as pd
from pandas_gbq import read_gbq
import plotly.express as px
import plotly.offline as pyo
import plotly.io as pio

pio.renderers.default = 'notebook'  # or 'jupyterlab' if using JupyterLab


## Libraries

In [24]:
NODE_DOMAIN_MAP = {
	"GABMKJM6I25XI4K7U6XWMULOUQIQ27BCTMLS6BYYSOWKTBUXVRJSXHYQ": "Stellar Development Foundation",
	"GCGB2S2KGYARPVIA37HYZXVRM2YZUEXA6S33ZU5BUDC6THSB62LZSTYH": "Stellar Development Foundation",
	"GCM6QMP3DLRPTAZW2UZPCPX2LF3SXWXKPMP3GKFZBDSF3QZGV2G5QSTK": "Stellar Development Foundation",
	"GAK6Z5UVGUVSEK6PEOCAYJISTT5EJBB34PN3NOLEQG2SUKXRVV2F6HZY": "SatoshiPay",
	"GBJQUIXUO4XSNPAUT6ODLZUJRV2NPXYASKUBY4G5MYP3M47PCVI55MNT": "SatoshiPay",
	"GC5SXLNAM3C4NMGK2PXK4R34B5GNZ47FYQ24ZIBFDFOCU6D4KBN4POAE": "SatoshiPay",
	"GCFONE23AB7Y6C5YZOMKUKGETPIAJA4QOYLS5VNS4JHBGKRZCPYHDLW7": "LOBSTR",
	"GCB2VSADESRV2DDTIVTFLBDI562K6KE3KMKILBHUHUWFXCUBHGQDI7VL": "LOBSTR",
	"GD5QWEVV4GZZTQP46BRXV5CUMMMLP4JTGFD7FWYJJWRL54CELY6JGQ63": "LOBSTR",
	"GA7TEPCBDQKI7JQLQ34ZURRMK44DVYCIGVXQQWNSWAEQR6KB4FMCBT7J": "LOBSTR",
	"GA5STBMV6QDXFDGD62MEHLLHZTPDI77U3PFOD2SELU5RJDHQWBR5NNK7": "LOBSTR",
	"GAAV2GCVFLNN522ORUYFV33E76VPC22E72S75AQ6MBR5V45Z5DWVPWEU": "Blockdaemon Inc.",
	"GAVXB7SBJRYHSG6KSQHY74N7JAFRL4PFVZCNWW2ARI6ZEKNBJSMSKW7C": "Blockdaemon Inc.",
	"GAYXZ4PZ7P6QOX7EBHPIZXNWY4KCOBYWJCA4WKWRKC7XIUS3UJPT6EZ4": "Blockdaemon Inc.",
	"GBLJNN3AVZZPG2FYAYTYQKECNWTQYYUUY2KVFN2OUKZKBULXIXBZ4FCT": "Public Node",
	"GCIXVKNFPKWVMKJKVK2V4NK7D4TC6W3BUMXSIJ365QUAXWBRPPJXIR2Z": "Public Node",
	"GCVJ4Z6TI6Z2SOGENSPXDQ2U4RKH3CNQKYUHNSSPYFPNWTLGS6EBH7I2": "Public Node",
	"GA7DV63PBUUWNUFAF4GAZVXU2OZMYRATDLKTC7VTCG7AU4XUPN5VRX4A": "Franklin Templeton",
	"GARYGQ5F2IJEBCZJCBNPWNWVDOFK7IBOHLJKKSG2TMHDQKEEC6P4PE4V": "Franklin Templeton",
	"GCMSM2VFZGRPTZKPH5OABHGH4F3AVS6XTNJXDGCZ3MKCOSUBH3FL6DOB": "Franklin Templeton",
	"GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7ZZAALMLNUVN": "Whalestack LLC",
	"GADLA6BJK6VK33EM2IDQM37L5KGVCY5MSHSHVJA4SCNGNUIEOTCR6J5T": "Whalestack LLC",
	"GAZ437J46SCFPZEDLVGDMKZPLFO77XJ4QVAURSJVRZK2T5S7XUFHXI2Z": "Whalestack LLC",
}

def get_longest_leaders(df, filter_last_five_min=True):
    df['close_at'] = pd.to_datetime(df['close_at'])

    # Step 1: Filter last five minute worth of data if arg set to true
    if filter_last_five_min:
        current_time = pd.Timestamp.now()
        five_minutes_ago = current_time - pd.Timedelta(minutes=5)
        filtered_df = df[df['close_at'] >= five_minutes_ago]
    else:
        filtered_df = df

    # Step 2: Assign row numbers based on closed_at
    filtered_df = filtered_df.sort_values('close_at')
    filtered_df['rn'] = range(1, len(filtered_df) + 1)

    # Step 3: Create a grouping identifier (grp)
    filtered_df['grp'] = filtered_df['rn'] - filtered_df.groupby('node_id')['rn'].transform(lambda x: x.rank(method='first'))

    # Step 4: Get start and end time for each group
    windowed_data = filtered_df.groupby(['node_id', 'grp']).agg(start_time=('close_at', 'min'),
                                                           end_time=('close_at', 'max')).reset_index()

    # Step 5: Calculate continuous_time and validator_frequency
    windowed_data['continuous_time'] = (windowed_data['end_time'] - windowed_data['start_time']).dt.total_seconds()
    result = windowed_data.groupby(['continuous_time', 'node_id']).size().reset_index(name='validator_frequency')

    # Step 6: Sort the results
    result = result.sort_values(by='continuous_time', ascending=False)
    result['home_domain'] = result['node_id'].map(NODE_DOMAIN_MAP)
    return result

def plot_chart(df, title, filename):
    fig = px.bar(
        df,
        x='continuous_time',
        y='validator_frequency',
        color='node_id',
        title=title,
        labels={'continuous_time': 'Continuous Time (seconds)', 'validator_frequency': 'Frequency'},
        text='home_domain'
    )

    # Update layout for grouping
    fig.update_layout(barmode='group')

    # Show the plot
    fig.write_html(f"{filename}.html")

## Full history

In [25]:
nodes = NODE_DOMAIN_MAP.keys()
nodes_string = ",".join(f"'{item}'" for item in nodes)
query = f"""
  SELECT
    hl.node_id as node_id,
    hl.closed_at AS close_at
  FROM crypto-stellar.crypto_stellar.history_ledgers AS hl
  WHERE hl.closed_at BETWEEN '2024-01-01 00:00:00 UTC' AND '2025-01-01 00:00:00 UTC'
  AND hl.node_id in ({nodes_string})
"""
full_df = read_gbq(query, project_id='crypto-stellar')
full_df['close_at'] = full_df['close_at'].dt.tz_localize(None)
result_df = get_longest_leaders(full_df, filter_last_five_min=False)
plot_chart(result_df[result_df["continuous_time"] >= 20], title="Nodes leading for greater than equal to 20 seconds", filename="full_history")

Downloading: 100%|█████████████████████████████████████████████████████████████████|


## Live data

In [ ]:
#  Socket to talk to server
context = zmq.Context()
socket = context.socket(zmq.SUB)

print("Collecting validator info from pipeline ...")
socket.connect("tcp://127.0.0.1:5555")
socket.subscribe("")

cur_data_vals = []

while True:

    message = socket.recv()
    json_object = json.loads(message)
    json_formatted_str = json.dumps(json_object, indent=2)
    print(f"Validator info:\n\n{json_formatted_str}")

    cur_data_vals.append(json_object)
    cur_df = pd.DataFrame(cur_data_vals)
    cur_df.rename(columns={'close_time': 'close_at'}, inplace=True)
    cur_df['close_at'] = pd.to_datetime(cur_df['close_at'], unit='s')
    cur_res_df = get_longest_leaders(cur_df)

    plot_chart(cur_res_df, title="Nodes leading for last 5 minutes", filename="live_data")
    print("Update live data chart")


Validator info:

{
  "sequence_number": "53975556",
  "node_id": "GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7ZZAALMLNUVN",
  "signature": "d9DzJApf6CIMKIlU1rI/9Ru7fXI1Z1tD62ge3BJEDWQIFQ2uiLNDk+AycT+WLUO5RS2KetQdvTv10SJhQen+DQ==",
  "name": "Whalestack LLC",
  "close_time": 1729103944
}
Update live data chart
Validator info:

{
  "sequence_number": "53975557",
  "node_id": "GABMKJM6I25XI4K7U6XWMULOUQIQ27BCTMLS6BYYSOWKTBUXVRJSXHYQ",
  "signature": "w8riuIyAK4xfv7e4boINU/DeiA0FiDREeNN8c+D1VsL9rJPVq31/hnlJ4z77ylZgBu+0kUb7WNGuFGdQDoVdBQ==",
  "name": "Stellar Development Foundation",
  "close_time": 1729103950
}
Update live data chart
Validator info:

{
  "sequence_number": "53975558",
  "node_id": "GA7DV63PBUUWNUFAF4GAZVXU2OZMYRATDLKTC7VTCG7AU4XUPN5VRX4A",
  "signature": "BmgYWGn9kyl8vWIjmrDUHC1etYQIjdGY4gMPvePoWWVmigOMR+HWSB6D1ME41emcT9jJDQ+1hCiPiZrFb5WtAw==",
  "name": "Franklin Templeton",
  "close_time": 1729103956
}
Update live data chart
Validator info:

{
  "sequence_number": "539